### Tuning the LSTM model ... 

File copied from "REAL" model tuning for EOF on 8/4/2025.  

In [1]:
##import cell makes its appearance once again...
%matplotlib inline
##so-called "math" related imports
#from netCDF4 import Dataset as ncread
import numpy as np
import pandas as pd
import math
from random import seed
from random import randint
from random import sample
from scipy.ndimage import gaussian_filter
from sklearn.metrics import brier_score_loss
import xarray as xr

import pickle

##plotting related imports
import matplotlib.pyplot as plt

#import tensorflow/keras related files
import tensorflow as tf    
#tf.compat.v1.disable_v2_behavior() # <-- HERE !

tf.device('/physical_device:GPU:0')

from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras import Input
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras import regularizers
import tensorflow.keras.backend as K
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.layers import Dropout, Activation, Reshape, Flatten, LSTM, Dense, Dropout, Embedding, Bidirectional, GRU
from tensorflow.keras import Sequential
from tensorflow.keras import initializers, regularizers
from tensorflow.keras import optimizers
from tensorflow.keras import constraints
from tensorflow.keras.layers import Layer, InputSpec

#tf.compat.v1.disable_eager_execution()
#import investigate

import keras_tuner as kt

2025-05-30 22:44:46.485005: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-05-30 22:44:46.516765: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9373] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-05-30 22:44:46.516786: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-05-30 22:44:46.517793: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1534] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-05-30 22:44:46.523112: I tensorflow/core/platform/cpu_feature_guar

In [2]:
print(tf.__version__)

2.15.0


In [5]:
#load input data, it is max/min standardized, NaNs removed
#stratospheric polar vortex ellipse diagnostics from Fernandez et al 2025 (in review)
#comparison here may be a little muddled because I am limitied in the days that I can observe, however, I am currently working(7/8/2025) on calculating the diagnostics for the full dataset
#this will be fixed. 

infile = open("../data/nolag_extendedanom_input.p", 'rb') 
nolag_input = pickle.load(infile)
infile.close()

nolag_input.shape

In [ ]:
idx = 149
#switch to 149 for normal. 

inp1 = np.empty((62,idx,8)) #create new input array, 14 day lag. 

shift = 0
#0 if normal
#this indicates setting the start date as November 2 rather than October 19

##reshaping to change lag in metrics for input
inp1[:,:,0] = nolag_input[:62,shift:,0] ##wind
inp1[:,:,1] = nolag_input[:62,shift:,1] ##ratio
inp1[:,:,2] = nolag_input[:62,shift:,2] ##latitude
inp1[:,:,3] = nolag_input[:62,shift:,3] ##longitude
inp1[:,:,4] = nolag_input[:62,shift:,4] ##size
inp1[:,:,5] = nolag_input[:62,shift:,5] ##ephi
inp1[:,:,6] = nolag_input[:62,shift:,6] ##gph
inp1[:,:,7] = nolag_input[:62,shift:,7] ##pv

inp=inp1.reshape(62*idx,8)
inp.shape

In [10]:
#load output data file.
#I can change this to represent any of the available temp regions. 
infile = open("../data/eur_anomtemps_reduced.p","rb",)
output = pickle.load(infile) 
infile.close()

In [11]:
output.shape

In [12]:
#convert to pandas dataframe
input = pd.DataFrame(inp)
#label columns of variables for input data
col_names = ['wind','rat','cenlat','cenlon','size','ephi','gph','pv']
input.columns = col_names

input##make pandas dataframe for RF
input = pd.DataFrame(inputvar)
input

,0,1,2,3,4,5,6,7,8,9,...,52,53,54,55,56,57,58,59,60,61
0,0.771931,-1.799955,1.313722,-1.630033,0.566290,1.589922,-0.172617,1.566368,0.659295,0.199667,...,-0.379306,1.641585,-0.743088,-0.070795,0.742423,0.454864,0.251855,0.795493,-0.417825,-1.029987
1,0.803999,-1.900528,1.245334,-1.692496,0.101666,2.042962,0.076518,1.506671,0.006843,0.238603,...,0.347120,1.752066,-0.985741,0.049346,0.764168,0.594407,0.432133,1.080915,-0.165571,-1.420320
2,0.872130,-2.032226,1.424598,-1.343837,0.161601,1.711995,0.503023,0.826614,-1.105172,0.270467,...,-0.585204,1.588777,-1.302961,0.097989,0.733291,0.651646,0.515277,0.912816,0.289325,-1.499576
3,0.867914,-2.054812,1.657540,-0.838510,0.746273,1.227320,0.679350,1.134812,-0.876215,0.232273,...,0.789144,1.273398,-1.416257,-0.048546,0.671334,0.746395,0.300846,0.709486,0.123002,-0.751547
4,0.787536,-2.011126,1.559197,-0.409157,1.239623,0.913107,0.641640,1.931937,-0.272880,0.127776,...,0.377190,1.043657,-1.369488,-0.219058,0.751703,1.057360,0.347496,0.404459,0.330464,0.215042
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8365,0.415159,-0.170777,-0.531934,-0.397453,1.102362,-0.267771,0.062068,1.148350,-2.241040,0.180624,...,-0.535923,1.136943,-0.416174,0.562862,-0.354710,-0.763675,-0.224326,0.846225,-1.504478,-0.403197
8366,0.484640,-0.232514,-0.597377,0.039627,0.837373,-0.276658,0.205001,1.321532,-2.205615,0.232497,...,-0.670336,1.115493,-0.641672,0.649173,-0.423895,-0.589386,-0.034272,0.739777,-1.244535,-0.653119
8367,0.558206,-0.163820,-0.476075,0.480464,0.879917,-0.589342,-0.360315,1.203000,-2.164044,0.126000,...,-1.172690,0.839839,-1.031493,0.910849,-0.425116,-0.327954,0.842602,1.100344,-0.612106,-0.448852
8368,0.591930,0.343880,-0.538673,1.014618,0.200438,-0.391132,-0.570871,1.115725,-1.033758,0.161263,...,-0.749049,0.472475,-1.311037,0.590795,-0.424628,-0.503666,0.747163,1.328334,-0.101280,-0.280176


In [ ]:
inp2 = input.values.reshape(62,149,8)
inp2.shape

In [13]:
##need to change this based on lag 
## 14-days = [:, 10:] 10 day window rather than 14
## 20-days = [:, 16:]
## 30-days = [:, 26:]

# 139
# 133
# 123

## 14-days = [:, 14:] 14 day window
## 20-days = [:, 20:]
## 30-days = [:, 30:]

# 135
# 129
# 119

lead = 135

temp = output.reshape(62, 149)
temp = temp[:, 14:]
print(temp.shape)

temp_flat = temp.flatten()

(62, 135)

In [19]:
#create timeseries data arrays for PCs 
#was 122 and 14 for the 14 day window. 

new_input =  np.empty((62,lead,14,8))
new_output = np.empty((62,lead))

for i in range(0,62):
    for j in range(0,lead):
        new_input[i,j,:,:] = inp2[i,j:j+14,:]
        new_output[i,j] = temp[i,j]

In [20]:
new_input = np.array(new_input)
new_output = np.array(new_output)
print(new_input.shape)
print(new_output.shape)

(62, 121, 14, 10)
(62, 121)


In [21]:
frac_ind = round((62*lead)*0.15)
frac_end = round((62*lead)-frac_ind)
frac_end

6377

In [22]:
frac_ind

1125

In [23]:
##Set X_all and Y_all datasets
X_all = np.copy(new_input.reshape((62*lead),14,8))
Y_all = np.copy(new_output.flatten())

##training data partition out
X_tri = X_all[:frac_end,:]
Y_tri = Y_all[:frac_end]

#testing data partition out
X_tes = X_all[frac_end:,:]
Y_tes = Y_all[frac_end:]

#Convert the Y array into a categorical array. This means we will create one-hot vector labels for all of the inputs.
# The one-hot vectors have an index for each possible output category (two in our case)
# A "1" is put in the index corresponding to the category to which the sample belongs
Y_all = keras.utils.to_categorical(Y_all)
Y_tri = keras.utils.to_categorical(Y_tri)
Y_tes= keras.utils.to_categorical(Y_tes)

In [24]:
Y_all.shape

(7502, 2)

In [25]:
##checking my data for NaN of Infs because I need to make sure this doesn't cause
#the model to throw back no loss

if np.any(np.isnan(X_all)) or np.any(np.isinf(X_all)):
    print("NaN or Inf values found in X_all!")

if np.any(np.isnan(Y_all)) or np.any(np.isinf(Y_all)):
    print("NaN or Inf values found in Y_all!")

### Begin establishing specifics of model.

In [26]:
##checking to make sure shape was properly one hot encoded
Y_all.shape

(7502, 2)

In [27]:
##class weight creator for the instance where now I have a four dimensional output array 
def class_weight_creator(Y):
    class_dict = {}
    Y_reshaped = Y.reshape(-1, Y.shape[-1])
    weights = np.max(np.sum(Y_reshaped, axis=0)) / np.sum(Y_reshaped, axis=0)
    for i in range(Y.shape[-1] ):
        class_dict[i] = weights[i]
        
    return class_dict

In [28]:
##number of input nodes
numb_int = X_all.shape[1:]

##fraction of training data
X_validation = X_tri[0:frac_ind]
Y_validation = Y_tri[0:frac_ind]
        
X_train = X_tri[frac_ind:len(X_tri)]
Y_train = Y_tri[frac_ind:len(Y_tri)]

X_test = X_tes
Y_test = Y_tes

In [29]:
Y_train.shape

(5252, 2)

In [30]:
numb_int

(14, 10)

In [31]:
##do the class_dict weights
class_weight = class_weight_creator(Y_train)
class_weight

{0: 1.0, 1: 2.1206179}

### Model Architecture. 

In [32]:
######
batch_size = 64 #The number of samples the network sees before it backpropagates (batch size)
epochs = 30 #The number of times the network will loop through the entire dataset (epochs)
shuffle = True #Set whether to shuffle the training data so the model doesn't see it sequentially 
verbose = 2 #Set whether the model will output information when trained (0 = no output; 2 = output accuracy every epoch)

In [33]:
def tune_LSTM(hp): 
    ##set ranges for hyperparameters
    n1 = hp.Int('n1', min_value=32, max_value=160, step=32)
    n2 = hp.Int('n2', min_value=4, max_value=32, step=4)
    n3 = hp.Int('n3', min_value=8, max_value=32, step=8)
    
    reg1 = hp.Float('reg1', min_value=0.001, max_value=0.9, step=0.01)
    reg2 = hp.Float('reg2', min_value=0.001, max_value=0.9, step=0.01)
    reg3 = hp.Float('reg3', min_value=0.001, max_value=0.9, step=0.01)
    
    
    input_tensor = Input(shape=(14, 8))
    layer1 = layers.RNN(
        layers.LSTMCell(n1, activation='tanh', use_bias=True,
                          kernel_initializer='glorot_uniform',
                          kernel_regularizer=keras.regularizers.l2(l2= reg1)),
        return_sequences=True)(input_tensor)
    layer2 = layers.RNN(
        layers.LSTMCell(n2, activation='tanh', use_bias=True,
                          kernel_initializer='glorot_uniform',
                          kernel_regularizer=keras.regularizers.l2(l2= reg2)),
                        return_sequences=False)(layer1)
    layer3 = layers.Dense(n3, activation='relu',use_bias=True,
                          kernel_initializer='he_normal',bias_initializer='he_normal',
                          kernel_regularizer=keras.regularizers.l2(l2=reg3))(layer2)

    output_tensor = layers.Dense(2, activation='softmax',)(layer3)

    model = Model(input_tensor, output_tensor)
    
    lr = hp.Float('learning_rate', min_value=0.001, max_value=0.1)
    
    opt = tf.keras.optimizers.legacy.Adam(learning_rate=lr)
    #decay_rate = lr / epochs
    #momentum = 0.9

    model.compile(optimizer=opt,loss='categorical_crossentropy',metrics=[keras.metrics.categorical_accuracy],)
                            
    return model

In [34]:
##the LSTM layers will have EXACTLY the same regularizer

In [35]:
tf.keras.backend.clear_session()

In [36]:
###################################################

In [37]:
#intialize/setup the hyperband tuner
tuner = kt.RandomSearch(
    tune_LSTM, #model
    objective = [kt.Objective('val_loss', 'min')],#kt.Objective('loss', 'min'),kt.Objective('val_categorical_accuracy', 'max')],
    max_trials=1000, #number of iterations of tuning to run
    max_consecutive_failed_trials=3, #number of allowed failed trials
    directory = 'tuning',
    project_name='ellipse_lstm1'
)

Reloading Tuner from tuning/EOF_seus1/tuner0.json


In [38]:
stop_early = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5,restore_best_weights=True,verbose=1)

In [ ]:
##actual tuning process
tuner.search(X_train, Y_train, validation_data=(X_validation, Y_validation),
             batch_size=batch_size, epochs=epochs, shuffle=shuffle,
             class_weight = class_weight, callbacks=[stop_early])

Trial 946 Complete [00h 00m 17s]
multi_objective: 0.6452176570892334

Best multi_objective So Far: 0.6266786456108093
Total elapsed time: 06h 55m 26s

Search: Running Trial #947

Value             |Best Value So Far |Hyperparameter
32                |64                |n1
24                |32                |n2
24                |16                |n3
0.66              |0.46              |reg1
0.45              |0.04              |reg2
0.24              |0.47              |reg3
0.076533          |0.086637          |learning_rate

Epoch 1/50
83/83 [==============================] - 3s 22ms/step - loss: 3.4607 - categorical_accuracy: 0.5061 - val_loss: 0.6903 - val_categorical_accuracy: 0.6880
Epoch 2/50
83/83 [==============================] - 1s 18ms/step - loss: 0.9465 - categorical_accuracy: 0.4895 - val_loss: 0.6748 - val_categorical_accuracy: 0.6880
Epoch 3/50
83/83 [==============================] - 1s 17ms/step - loss: 0.9469 - categorical_accuracy: 0.4770 - val_loss: 0.6907 - v

In [ ]:
#Get the optimal hyperparameters
best_hps=tuner.get_best_hyperparameters(num_trials=1)[0]
best_hps.values

In [ ]:
# Build the model with the optimal hyperparameters and train it on the data for 50 epochs
lstm_model = tuner.hypermodel.build(best_hps)
history = lstm_model.fit(X_train,Y_train, validation_data=(X_validation, Y_validation),
                    batch_size=batch_size, epochs=epochs, shuffle=shuffle,class_weight = class_weight)


In [ ]:
train_loss=lstm_model.history.history['loss']
val_loss=lstm_model.history.history['val_loss']
    
cat_acc=lstm_model.history.history['categorical_accuracy']
val_acc=lstm_model.history.history['val_categorical_accuracy']
        
pred = lstm_model.predict(X_all)
pred_val = lstm_model.predict(X_validation)
pred_train = lstm_model.predict(X_train)
pred_test = lstm_model.predict(X_test)

# Look at the optimization history
fig, (ax1,ax2) = plt.subplots(1,2, figsize=plt.figaspect(0.25))
ax1.plot(train_loss, label='Training loss')
ax1.plot(val_loss, label='Validation loss')
ax1.set_yscale('log')
ax1.set_title('loss')
ax1.set_xlabel('Epoch')
ax1.set_ylabel('loss')
ax1.legend()

ax2.plot(cat_acc, label='Training ACC')
ax2.plot(val_acc, label='Validation ACC')
ax2.set_title('Accuracy')
ax2.set_xlabel('Epoch')
ax2.set_ylabel('Acc')
ax2.legend